In [5]:
# %load /home/mithrillion/default_imports.py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import style
style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline
import re
from collections import Counter
from functools import partial

import folium
import boto3
from time import sleep

ModuleNotFoundError: No module named 'folium'

In [ ]:
client = boto3.client('athena')

In [ ]:
def query_to_df(res):
    colnames = [x['Name'] for x in res['ResultSet']['ResultSetMetadata']['ColumnInfo']]
    typemap = {'bigint': np.int32, 'varchar': np.str, 'double': np.double, 'boolean': np.bool}
    coltypes = [typemap[x['Type']] for x in res['ResultSet']['ResultSetMetadata']['ColumnInfo']]
    rows = [[item['VarCharValue'] for item in row['Data']] for row in res['ResultSet']['Rows'][1:]]
    df = pd.DataFrame(rows, columns=colnames)
    for n, t in zip(colnames, coltypes):
        if t == np.bool:
            df[n] = (df[n] == 'true')
        df[n] = df[n].astype(t)
    return df

In [ ]:
def get_result(job):
    while (True):
        try:
            paginator = client.get_paginator('get_query_results')
            p = paginator.paginate(QueryExecutionId=job['QueryExecutionId'])
            return pd.concat([query_to_df(part) for part in p], axis=0)
        except Exception as e:
            if str(e)[-7:] == 'RUNNING':
                sleep(10)
            else:
                print(e)
                break

## Card Type Breakdown

In [ ]:
query = """
SELECT
  CardType,
  Card_SubType_Desc,
  Concession_Type,
  COUNT(DISTINCT CardID) Cnt
FROM
  samp0,
  card_types
WHERE
  CardType = Card_SubType_ID
GROUP BY
  CardType,
  Card_SubType_Desc,
  Concession_Type
ORDER BY
  Cnt DESC
"""
job = client.start_query_execution(
    QueryString=query,
    QueryExecutionContext={'Database': 'melb-dthon'},
    ResultConfiguration={
        'OutputLocation':
        's3://aws-athena-query-results-796452490322-us-east-2/',
        'EncryptionConfiguration': {
            'EncryptionOption': 'SSE_S3',
        }
    })

In [ ]:
job

In [ ]:
card_type_breakdown = get_result(job)

In [ ]:
card_type_breakdown.head()